# Install Library

In [ ]:
# !pip install sklearn_crfsuite
!pip install -U numpy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 87.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.


# Import Library

In [ ]:
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
# from sklearn_crfsuite import CRF
# from sklearn_crfsuite import metrics
# from sklearn_crfsuite import scorers
from collections import Counter
import nltk


# Import Dataset

In [ ]:
nltk.download("treebank")
nltk.download("universal_tagset")

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [ ]:
tagged_sentence = nltk.corpus.treebank.tagged_sents(tagset="universal")

In [ ]:
print("Number of Tagged Sentences",len(tagged_sentence))
tagged_words = [tup for sent in tagged_sentence for tup in sent]
print("Total Number of Tagged words", len(tagged_words))
vocab = set([word for word, tag in tagged_words])
print("Vocabulary of the Corpus",len(vocab))
tags = set([tag for word,tag in tagged_words])
print("Number of Tags in the Corpus",len(tags))


Number of Tagged Sentences 3914
Total Number of Tagged words 100676
Vocabulary of the Corpus 12408
Number of Tags in the Corpus 12


In [ ]:
train_set, test_set = train_test_split(tagged_sentence, test_size=0.2, random_state=1234)
print("Number of Sentences in Training Data",len(train_set))
print("Number of Sentences in Test Data",len(test_set))


Number of Sentences in Training Data 3131
Number of Sentences in Test Data 783


# Machine Leaning

Define the feature function. The following features can be used:
1. Is the first letter capitalized?
2. Is it the first word in the sentence?
3. Is it the last word?
4. What is the prefix of the word?
5. What is the suffix of the word?
6. Is the complete word capitalized
7. What is the previous word?
8. What is the next word?
9. Is it numeric?
10. Is it alphaunumeric?
11. Is there a hyphen in the word?

In [ ]:
# work feature
def features(sentence,index):
  return {
      'is_first_capital':int(sentence[index][0].isupper()),
      'is_first_word':int(index==0),
      'is_last_word':int(index==len(sentence)-1),
      'is_complete_capital':int(sentence[index].upper()==sentence[index]),
      'prev_word':'' if index==0 else sentence[index-1],
      'next_word':'' if index==len(sentence)-1 else sentence[index+1],
      'is_numeric':int(sentence[index].isdigit()),
      'is_alphanumeric': int(bool((re.match('^(?=.*[0-9]$)(?=.*[a-zA-z])',sentence[index])))),
      'prefix_1':sentence[index][0],
      'prefix_2':sentence[index][:2],
      'prefix_3':sentence[index][:3],
      'prefix_4':sentence[index][:4],
      'suffix_1':sentence[index][-1],
      'suffix_2':sentence[index][-2:],
      'suffix_3':sentence[index][-3:],
      'suffix_4':sentence[index][-4:],
      'word_has_hypen': 1 if '-' in sentence[index] else 0
  }

def untag(sentence):
  return [word for word,tag in sentence]
# prepare data
def prepareData(tagged_sentences):
  X,y=[],[]
  for sentences in tagged_sentences:
    X.append([features(untag(sentences), index) for index in range(len(sentences))])
    y.append([tag for word, tag in sentences])
  return X,y

In [ ]:
X_train, y_train=prepareData(train_set)
X_test,y_test=prepareData(test_set)

In [ ]:
# only on sequence
X_train[0]

[{'is_first_capital': 1,
  'is_first_word': 1,
  'is_last_word': 0,
  'is_complete_capital': 0,
  'prev_word': '',
  'next_word': 'Wall',
  'is_numeric': 0,
  'is_alphanumeric': 0,
  'prefix_1': 'O',
  'prefix_2': 'On',
  'prefix_3': 'On',
  'prefix_4': 'On',
  'suffix_1': 'n',
  'suffix_2': 'On',
  'suffix_3': 'On',
  'suffix_4': 'On',
  'word_has_hypen': 0},
 {'is_first_capital': 1,
  'is_first_word': 0,
  'is_last_word': 0,
  'is_complete_capital': 0,
  'prev_word': 'On',
  'next_word': 'Street',
  'is_numeric': 0,
  'is_alphanumeric': 0,
  'prefix_1': 'W',
  'prefix_2': 'Wa',
  'prefix_3': 'Wal',
  'prefix_4': 'Wall',
  'suffix_1': 'l',
  'suffix_2': 'll',
  'suffix_3': 'all',
  'suffix_4': 'Wall',
  'word_has_hypen': 0},
 {'is_first_capital': 1,
  'is_first_word': 0,
  'is_last_word': 0,
  'is_complete_capital': 0,
  'prev_word': 'Wall',
  'next_word': 'men',
  'is_numeric': 0,
  'is_alphanumeric': 0,
  'prefix_1': 'S',
  'prefix_2': 'St',
  'prefix_3': 'Str',
  'prefix_4': 'Stre'

In [ ]:
y_train[0]

['ADP',
 'NOUN',
 'NOUN',
 'NOUN',
 'CONJ',
 'NOUN',
 'VERB',
 'ADP',
 'ADJ',
 'NOUN',
 '.',
 'X',
 'VERB',
 'NUM',
 'DET',
 'ADV',
 'ADV',
 'PRON',
 'VERB',
 'ADP',
 'NOUN',
 'X',
 '.']

## Train model with CRF

In [ ]:
crf = CRF(
    algorithm="lbfgs",
    c1=0.01,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
# crf.fit(X_train, y_train)

try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

In [ ]:
y_pred=crf.predict(X_test)

In [ ]:
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=crf.classes_)

0.9740622075583344

In [ ]:
y_pred_train=crf.predict(X_train)
metrics.flat_f1_score(y_train, y_pred_train, average='weighted', labels=crf.classes_)

0.9962782771443118

# Deep Learning

## Import Dataset

In [ ]:
nltk.download('treebank')
tagged_sentences = nltk.corpus.treebank.tagged_sents()[:100]

print(tagged_sentences[0])
print("Tagged sentences: ",len(tagged_sentences))
print("Tagged words:",len(nltk.corpus.treebank.tagged_words()))

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]
Tagged sentences:  100
Tagged words: 100676


In [ ]:
import numpy as np

sentences, sentence_tags = [], []
for tagged_sentence in tagged_sentences:
  sentence, tags = zip(*tagged_sentence)
  sentences.append(np.array(sentence))
  sentence_tags.append(np.array(tags))
print(sentences[5])
print(sentence_tags[5])

['Lorillard' 'Inc.' ',' 'the' 'unit' 'of' 'New' 'York-based' 'Loews'
 'Corp.' 'that' '*T*-2' 'makes' 'Kent' 'cigarettes' ',' 'stopped' 'using'
 'crocidolite' 'in' 'its' 'Micronite' 'cigarette' 'filters' 'in' '1956'
 '.']
['NNP' 'NNP' ',' 'DT' 'NN' 'IN' 'JJ' 'JJ' 'NNP' 'NNP' 'WDT' '-NONE-' 'VBZ'
 'NNP' 'NNS' ',' 'VBD' 'VBG' 'NN' 'IN' 'PRP$' 'NN' 'NN' 'NNS' 'IN' 'CD'
 '.']


## Train_test_split

In [ ]:
(train_sentences,
 test_sentences,
 train_tags,
 test_tags) = train_test_split(sentences, sentence_tags, test_size=0.2)
print(len(train_sentences),len(test_sentences),len(train_tags),len(test_tags))

80 20 80 20


transfer character to number for train in deep learning

In [ ]:
words, tags = set([]), set([])

for s in train_sentences:
  for w in s:
    words.add(w.lower())

  for ts in train_tags:
    for t in ts:
      tags.add(t)

  word2index = {w: i + 2 for i, w in enumerate(list(words))} # start in index 2 because start padding index 1 and OOVs index 2
  word2index['-PAD-'] = 0 # The special value used for padding to add 0 for nan values caused by unequal sentences (long sentences do not add 0, short sentences produce nan values, must add 0)
  word2index['-OOV-'] = 1 # The special value used for OOVs(out of vocab) Words that are not in our dataset are new words.

  tag2index = {t: i + 1 for i, t in enumerate(list(tags))} # start in index 1 bacause start padding index = 0
  tag2index['-PAD-'] = 0 # same case in sentece add 0 for nan values caused by unequal sentences (long sentences do not add 0, short sentences produce nan values, must add 0)

In [ ]:
len(tag2index)
test_tags

[array(['IN', 'CD', 'NNS', 'IN', 'DT', 'NN', 'WDT', '-NONE-', 'VBD', 'NN',
        'IN', 'DT', 'NNP', 'NNS', 'VBD', 'VBN', '-NONE-', 'TO', 'NN', 'IN',
        'DT', 'CD', '.'], dtype='<U6'),
 array(['WRB', 'PRP', 'VBZ', 'NN', 'IN', 'PRP$', 'JJ', 'NN', '-NONE-', ',',
        'DT', 'NN', 'POS', 'VBG', 'NNS', 'RB', 'VBP', 'RP', 'TO', 'DT',
        'JJ', 'NNS', 'IN', 'NN', 'NNS', 'IN', 'NNP', 'NNP', 'CC', 'NNP',
        'NNP', '.'], dtype='<U6'),
 array(['PRP', 'VBZ', 'CD', 'NNS', 'CC', 'VBZ', 'JJ', 'NN', 'IN', 'IN',
        '$', 'CD', 'CD', '-NONE-', '.'], dtype='<U6'),
 array(['NN', 'NNS', 'VBD', '-NONE-', 'NNS', 'IN', 'DT', 'NN', 'IN', 'DT',
        'NN', 'MD', 'VB', 'IN', 'DT', 'NN', 'NN', 'IN', '$', 'CD', 'CD',
        '-NONE-', '.'], dtype='<U6'),
 array(['IN', 'NNP', ',', 'DT', 'NNP', 'NNP', 'NNP', 'VBD', 'DT', 'JJ',
        'NN', 'IN', 'RB', 'DT', 'NNS', 'IN', 'NN', '.'], dtype='<U3'),
 array(['IN', 'CD', 'NNS', 'WP', '-NONE-', 'VBD', 'RB', 'IN', 'DT', 'NN',
        ',', 'CD', '-NO

In [ ]:
# map word to sentence in train and test
# if it has a word change every word to lower case. if do not has word in dictionary set to oov
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []

for s in train_sentences:
  s_int = []
  for w in s:
    try:
      s_int.append(word2index[w.lower()])
    except KeyError:
      s_int.append(word2index['-00V-'])
  train_sentences_X.append(s_int)

  for s in test_sentences:
    s_int = []
    for w in s:
      try:
        s_int.append(word2index[w.lower()])
      except KeyError:
        s_int.append(word2index['-OOV-'])
  test_sentences_X.append(s_int)

  for s in train_tags:
    train_tags_y.append([tag2index[t] for t in s])

  for s in test_tags:
    test_tags_y.append([tag2index[t] for t in s])

print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])
print(len(train_sentences_X),len(test_sentences_X),len(train_tags_y),len(test_tags_y))

[748, 321, 567, 438, 434, 524, 461, 499, 80, 217, 78, 151, 416, 136, 205]
[78, 688, 321, 30, 370, 78, 293, 33, 1, 266, 1, 130, 695, 94, 113, 239, 1, 370, 1, 699, 78, 1, 528, 1, 457, 1, 398, 334, 266, 667, 329, 1, 1, 398, 1, 528, 525, 488, 1, 1, 380, 528, 113, 1, 370, 1, 714, 78, 1, 370, 1, 1, 370, 792, 205]
[19, 6, 33, 8, 8, 14, 17, 34, 1, 34, 33, 8, 17, 17, 22]
[34, 5, 1, 34, 33, 17, 13, 10, 7, 17, 34, 33, 19, 1, 7, 12, 10, 23, 17, 34, 33, 5, 22]
80 80 6400 1600


In [ ]:
# how many word in our longest sentence
MAX_LENGTH = len(max(train_sentences_X, key=len))
print(MAX_LENGTH)

47


In [ ]:
# from keras.preprocessing.sequence import pad_sequences
# from keras_preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences

# every sentences have same length the longest sentences because add padding(0) for same length
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding = 'post')

print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])
print(len(train_sentences_X),len(test_sentences_X),len(train_tags_y),len(test_tags_y))


[748 321 567 438 434 524 461 499  80 217  78 151 416 136 205   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0]
[  1 266   1 130 695  94 113 239   1 370   1 699  78   1 528   1 457   1
 398 334 266 667 329   1   1 398   1 528 525 488   1   1 380 528 113   1
 370   1 714  78   1 370   1   1 370 792 205]
[19  6 33  8  8 14 17 34  1 34 33  8 17 17 22  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[34  5  1 34 33 17 13 10  7 17 34 33 19  1  7 12 10 23 17 34 33  5 22  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
80 80 6400 1600


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from tensorflow.keras.optimizers import Adam




model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH)))
model.add(Embedding(len(word2index),128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 47, 128)           102912    
                                                                 
 bidirectional_1 (Bidirectio  (None, 47, 512)          788480    
 nal)                                                            
                                                                 
 time_distributed_1 (TimeDis  (None, 47, 36)           18468     
 tributed)                                                       
                                                                 
 activation_1 (Activation)   (None, 47, 36)            0         
                                                                 
Total params: 909,860
Trainable params: 909,860
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# one hot encoding in tag(label)
def to_categorical(sequences, categories):
  cat_sequences = []
  for s in sequences:
    cats = []
    for item in s:
      cats.append(np.zeros(categories))
      cats[-1][item] = 1.0
    cat_sequences.append(cats)
  return np.array(cat_sequences)

In [ ]:
cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))
print(cat_train_tags_y[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [ ]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=128, epochs=40, validation_split=0.2)

Epoch 1/40
1/1 [==============================] - 4s 4s/step - loss: 3.6059 - accuracy: 0.0180 - val_loss: 3.5309 - val_accuracy: 0.4468
Epoch 2/40
1/1 [==============================] - 0s 43ms/step - loss: 3.5231 - accuracy: 0.5037 - val_loss: 3.4559 - val_accuracy: 0.4455
Epoch 3/40
1/1 [==============================] - 0s 44ms/step - loss: 3.4367 - accuracy: 0.5010 - val_loss: 3.3618 - val_accuracy: 0.4455
Epoch 4/40
1/1 [==============================] - 0s 41ms/step - loss: 3.3277 - accuracy: 0.5010 - val_loss: 3.2220 - val_accuracy: 0.4455
Epoch 5/40
1/1 [==============================] - 0s 44ms/step - loss: 3.1644 - accuracy: 0.5010 - val_loss: 2.9806 - val_accuracy: 0.4455
Epoch 6/40
1/1 [==============================] - 0s 42ms/step - loss: 2.8781 - accuracy: 0.5010 - val_loss: 2.5908 - val_accuracy: 0.4455
Epoch 7/40
1/1 [==============================] - 0s 41ms/step - loss: 2.4135 - accuracy: 0.5010 - val_loss: 2.3784 - val_accuracy: 0.4455
Epoch 8/40
1/1 [=============

In [ ]:
# scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))
# print(f"{model.metrics_names[1]}: {scores[1]*100}")

In [ ]:
# # len(to_categorical(test_tags_y, len(tag2index)))
# len(tag2index)

36

In [ ]:
test_samples = [
    "running is very important for me".split(" "),
    "I was running every day for a month".split(" ")
]
print(test_samples)

[['running', 'is', 'very', 'important', 'for', 'me'], ['I', 'was', 'running', 'every', 'day', 'for', 'a', 'month']]


In [ ]:
test_samples_X = []
for s in test_samples:
  s_int = []
  for w in s:
    try:
      s_int.append(word2index[w.lower()])
    except KeyError:
      s_int.append(word2index['-OOV-'])
  test_samples_X.append(s_int)
test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
print(test_samples_X)

[[  1 321 659   1 699   1   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0]
 [  1 165   1   1 746 699 113   1   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0]]


In [ ]:
predictions = model.predict(test_samples_X)
print(predictions, predictions.shape)

1/1 [==============================] - 1s 690ms/step
[[[5.89950522e-03 6.09556548e-02 3.73745593e-03 ... 8.66259709e-02
   1.11601532e-01 4.15267050e-03]
  [6.41434779e-03 6.17145039e-02 3.67763499e-03 ... 8.67674947e-02
   1.11084647e-01 4.08719387e-03]
  [6.87334500e-03 6.22265637e-02 3.67832067e-03 ... 8.52579549e-02
   1.10751875e-01 4.04979428e-03]
  ...
  [9.98511732e-01 7.30851070e-06 1.27917389e-07 ... 2.13833414e-06
   3.36732319e-06 3.03101686e-08]
  [9.98366416e-01 8.22304628e-06 1.69725240e-07 ... 2.43918748e-06
   3.82869894e-06 3.94911943e-08]
  [9.98131931e-01 9.88189095e-06 2.33284240e-07 ... 2.98095370e-06
   4.71358817e-06 5.36617755e-08]]

 [[5.54236863e-03 6.04543686e-02 3.77471955e-03 ... 8.71446505e-02
   1.11965604e-01 4.17072186e-03]
  [5.75029477e-03 6.10807426e-02 3.71698337e-03 ... 8.65530074e-02
   1.12243481e-01 4.13642637e-03]
  [6.11612340e-03 6.08664267e-02 3.67016811e-03 ... 8.63808021e-02
   1.13005541e-01 4.04495141e-03]
  ...
  [9.98511851e-01 7.3079

In [ ]:
def logits_to_tokens(sequences,index):
  token_sequences = []
  for categorical_sequence in sequences:
    token_sequence = []
    for categorical in categorical_sequence:
      token_sequence.append(index[np.argmax(categorical)])
    token_sequences.append(token_sequences)

In [ ]:
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))

[['JJR', 'IN', 'DT', 'JJ', 'NNS', 'VBP', '-NONE-', 'DT', 'JJ', 'NN', 'VBZ', 'PRP', 'TO', 'VB', 'TO', 'DT', 'NN', 'NN', 'IN', 'JJ', 'NNS', '.']]


# NLTK

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
sentence = 'I like you'
token = nltk.word_tokenize(sentence)
token

['I', 'like', 'you']

In [ ]:
nltk.pos_tag(token)

[('I', 'PRP'), ('like', 'VBP'), ('you', 'PRP')]

# PyThaiNLP

In [ ]:
!pip install pythainlp[full]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 106.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 45.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.9/164.9 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 127.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.9/257.9 kB 30.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 82.9 

In [ ]:
import pythainlp
from pythainlp import sent_tokenize, word_tokenize

In [ ]:
text = "หัวหน้าฮงเป็นตัวอย่างของคนที่ดีเกินไปจนทำร้ายตัวเอง"

In [ ]:
sent = word_tokenize(text, engine='newmm')
print(sent)

['หัวหน้า', 'ฮง', 'เป็น', 'ตัวอย่าง', 'ของ', 'คน', 'ที่', 'ดี', 'เกินไป', 'จน', 'ทำร้าย', 'ตัวเอง']


In [ ]:
from pythainlp import Tokenizer
from pythainlp.util import Trie
from pythainlp.corpus.common import thai_words
words = ["ดีเกินไป"]
custom_words_list = set(thai_words())
custom_words_list.update(words)
trie = Trie(words=custom_words_list)
custom_tokenizer = Tokenizer(custom_dict=trie, engine='newmm')
sent = custom_tokenizer.word_tokenize(text)
print(sent)

['หัวหน้า', 'ฮง', 'เป็น', 'ตัวอย่าง', 'ของ', 'คน', 'ที่', 'ดีเกินไป', 'จน', 'ทำร้าย', 'ตัวเอง']


In [ ]:
from pythainlp.tag import pos_tag, pos_tag_sents
pos_tag(sent)

[('หัวหน้า', 'NCMN'),
 ('ฮง', 'NCMN'),
 ('เป็น', 'VSTA'),
 ('ตัวอย่าง', 'NCMN'),
 ('ของ', 'RPRE'),
 ('คน', 'NCMN'),
 ('ที่', 'PREL'),
 ('ดีเกินไป', 'VSTA'),
 ('จน', 'JSBR'),
 ('ทำร้าย', 'DONM'),
 ('ตัวเอง', 'PDMN')]

In [ ]:
pos_tag(sent, corpus="pud")

[('หัวหน้า', 'NOUN'),
 ('ฮง', 'PROPN'),
 ('เป็น', 'AUX'),
 ('ตัวอย่าง', 'NOUN'),
 ('ของ', 'ADP'),
 ('คน', 'NOUN'),
 ('ที่', 'DET'),
 ('ดีเกินไป', 'VERB'),
 ('จน', 'ADP'),
 ('ทำร้าย', 'ADP'),
 ('ตัวเอง', 'PRON')]

In [ ]:
pos_tag(sent, engine="unigram")

[('หัวหน้า', 'NCMN'),
 ('ฮง', ''),
 ('เป็น', 'VSTA'),
 ('ตัวอย่าง', 'NCMN'),
 ('ของ', 'RPRE'),
 ('คน', 'CNIT'),
 ('ที่', 'PREL'),
 ('ดีเกินไป', ''),
 ('จน', 'JSBR'),
 ('ทำร้าย', ''),
 ('ตัวเอง', 'PDMN')]